In [1]:
import pandas as pd

In [2]:
stints_2019 = pd.read_csv("..\\data\\stints_data_2019.csv", index_col=None)
stints_2020 = pd.read_csv("..\\data\\stints_data_2020.csv", index_col=None)
stints_2021 = pd.read_csv("..\\data\\stints_data_2021.csv", index_col=None)
stints_2022 = pd.read_csv("..\\data\\stints_data_2022.csv", index_col=None)
stints_2023 = pd.read_csv("..\\data\\stints_data_2023.csv", index_col=None)
stints_2024 = pd.read_csv("..\\data\\stints_data_2024.csv", index_col=None)

stints = pd.concat(
    [stints_2019, stints_2020, stints_2021, stints_2022, stints_2023, stints_2024]
)

stints = stints.astype(
    {
        "Driver": "category",
        "Stint": "int64",
        "Compound": "category",
        "StintLength": "int64",
        "Team": "category",
        "StartingPosition": "int64",
        "AirTemp": "float64",
        "Humidity": "float64",
        "Pressure": "float64",
        "Rainfall": "float64",
        "TrackTemp": "float64",
        "WindDirection": "float64",
        "WindSpeed": "float64",
        "mean_brake_time": "float64",
        "std_brake_time": "float64",
        "AvgSpeed": "float64",
        "StdSpeed": "float64",
        "AvgSpeedDelta": "float64",
        "StdSpeedDelta": "float64",
        "AvgGearChanges": "float64",
        "Year": "int64",
        "Circuit": "category",
    }
)

stints.drop(["Time"], axis=1, inplace=True, errors="ignore")

stints

,Driver,Stint,Compound,StintLength,Team,StartingPosition,AirTemp,Humidity,Pressure,Rainfall,...,WindSpeed,mean_brake_time,std_brake_time,AvgSpeed,StdSpeed,AvgSpeedDelta,StdSpeedDelta,AvgGearChanges,Year,Circuit
0,ALB,1,SOFT,14,Toro Rosso,13,23.477869,70.453279,1015.334426,0.0,...,1.166393,16.848500,2.609767,204.453924,9.638613,228.785714,27.103449,46.500000,2019,Australian Grand Prix
1,ALB,2,MEDIUM,43,Toro Rosso,13,23.477869,70.453279,1015.334426,0.0,...,1.166393,15.366744,1.481661,210.861675,3.012902,223.076478,11.134140,44.883721,2019,Australian Grand Prix
2,BOT,1,SOFT,23,Mercedes,2,23.477869,70.453279,1015.334426,0.0,...,1.166393,12.160522,1.770602,212.393517,7.822535,218.826087,22.160780,40.173913,2019,Australian Grand Prix
3,BOT,2,MEDIUM,35,Mercedes,2,23.477869,70.453279,1015.334426,0.0,...,1.166393,12.335229,1.054448,216.793137,2.787934,213.477143,6.962104,39.457143,2019,Australian Grand Prix
4,GAS,1,MEDIUM,37,Red Bull Racing,17,23.477869,70.453279,1015.334426,0.0,...,1.166393,15.714432,2.424948,209.376996,7.382757,234.907883,18.758009,40.918919,2019,Australian Grand Prix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,VER,1,MEDIUM,29,Red Bull Racing,4,26.768243,51.445946,1017.426351,0.0,...,1.900676,16.066103,1.878458,205.419774,9.531109,259.655172,17.201195,39.620690,2024,Abu Dhabi Grand Prix
748,VER,2,HARD,29,Red Bull Racing,4,26.768243,51.445946,1017.426351,0.0,...,1.900676,14.707172,3.306982,210.731920,9.665738,258.206897,13.425091,40.034483,2024,Abu Dhabi Grand Prix
749,ZHO,1,MEDIUM,12,Kick Sauber,15,26.768243,51.445946,1017.426351,0.0,...,1.900676,18.329667,2.409446,200.108530,14.604467,262.083333,23.286113,45.833333,2024,Abu Dhabi Grand Prix
750,ZHO,2,HARD,27,Kick Sauber,15,26.768243,51.445946,1017.426351,0.0,...,1.900676,16.529074,1.627520,207.185117,7.696528,250.592593,13.073182,41.814815,2024,Abu Dhabi Grand Prix


In [3]:
def transform_stints_df(df):
    """
    Transforms a DataFrame of stint data into a format where each row
    represents a single driver's complete race.

    This function groups all stints for a driver in a given race (identified by
    Driver, Year, and Circuit) into a single row, aggregating the data to provide
    a summary of the race strategy and performance.

    Args:
        df (pd.DataFrame): The input DataFrame containing stint-by-stint data.
                           It must include 'Driver', 'Year', 'Circuit', and 'Stint' columns.

    Returns:
        pd.DataFrame: A transformed DataFrame where each row is a unique race for a driver.
                      Columns for 'Compound' and 'StintLength' are aggregated into lists,
                      while other metrics are aggregated using appropriate measures (mean, first, sum).
    """
    # Ensure stints are in the correct order before aggregating into lists
    df_sorted = df.sort_values(["Driver", "Year", "Circuit", "Stint"])

    # Define the aggregation logic for each column.
    # We create lists for stint-specific data and aggregate others.
    agg_rules = {
        "Stint": "count",
        "Compound": lambda x: list(x),
        "StintLength": lambda x: list(x),
        "Team": "first",
        "StartingPosition": "first",
        "AirTemp": "mean",
        "Humidity": "mean",
        "Pressure": "mean",
        "Rainfall": "mean",
        "TrackTemp": "mean",
        "WindDirection": "mean",
        "WindSpeed": "mean",
        "mean_brake_time": "mean",
        "std_brake_time": "mean",
        "AvgSpeed": "mean",
        "StdSpeed": "mean",
        "AvgSpeedDelta": "mean",
        "StdSpeedDelta": "mean",
        "AvgGearChanges": "mean",
    }

    # Group by the unique race identifier and apply the aggregation rules
    strategy_df = (
        df_sorted.groupby(["Driver", "Year", "Circuit"], observed=True)
        .agg(agg_rules)
        .reset_index()
    )

    # Rename columns for clarity
    strategy_df.rename(
        columns={
            "Stint": "NumberOfStints",
            "Compound": "CompoundStrategy",
            "StintLength": "StintLengthStrategy",
        },
        inplace=True,
    )

    strategy_df = strategy_df[
        ~strategy_df["CompoundStrategy"].apply(lambda compounds: "UNKNOWN" in compounds)
    ]

    return strategy_df

In [4]:
stints_transformed = transform_stints_df(stints)
stints_transformed

,Driver,Year,Circuit,NumberOfStints,CompoundStrategy,StintLengthStrategy,Team,StartingPosition,AirTemp,Humidity,...,TrackTemp,WindDirection,WindSpeed,mean_brake_time,std_brake_time,AvgSpeed,StdSpeed,AvgSpeedDelta,StdSpeedDelta,AvgGearChanges
0,AIT,2020,Sakhir Grand Prix,4,"[MEDIUM, HARD, MEDIUM, SOFT]","[31, 23, 5, 28]",Williams,17,20.947244,58.169291,...,23.469291,122.755906,1.948031,11.550400,4.208429,184.819084,38.307985,247.199782,37.944436,35.335238
1,ALB,2019,Abu Dhabi Grand Prix,2,"[MEDIUM, HARD]","[13, 42]",Red Bull Racing,5,26.018045,64.471429,...,29.566165,221.097744,1.264662,20.155587,1.190986,190.146518,4.104201,246.565018,15.577114,48.759158
2,ALB,2019,Australian Grand Prix,2,"[SOFT, MEDIUM]","[14, 43]",Toro Rosso,13,23.477869,70.453279,...,41.313115,155.327869,1.166393,16.107622,2.045714,207.657799,6.325757,225.931096,19.118794,45.691860
3,ALB,2019,Austrian Grand Prix,2,"[MEDIUM, HARD]","[35, 35]",Toro Rosso,18,34.444068,17.965254,...,50.744068,188.855932,1.225424,13.635129,2.089451,219.187816,6.962654,240.657143,15.165511,38.714286
4,ALB,2019,Azerbaijan Grand Prix,2,"[SOFT, MEDIUM]","[12, 38]",Toro Rosso,11,19.763566,51.408527,...,39.654264,167.480620,1.249612,26.073193,2.424288,193.901751,7.891566,269.989031,16.208025,65.037281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2349,ZHO,2024,Qatar Grand Prix,4,"[MEDIUM, HARD, HARD, HARD]","[35, 1, 1, 20]",Kick Sauber,12,18.926490,57.251656,...,22.773510,105.649007,1.657616,27.345254,5.259709,175.615265,15.883737,229.368156,6.710474,50.978571
2350,ZHO,2024,Singapore Grand Prix,2,"[HARD, MEDIUM]","[34, 27]",Kick Sauber,20,30.786250,74.650000,...,36.445625,192.493750,0.955000,24.642124,1.980963,176.032457,5.536413,233.619826,12.504193,50.650871
2351,ZHO,2024,Spanish Grand Prix,3,"[SOFT, MEDIUM, HARD]","[9, 32, 24]",Kick Sauber,15,24.132468,63.441558,...,41.096104,207.370130,2.123377,17.048671,2.256758,202.696363,7.376482,225.560185,18.007868,36.703704
2352,ZHO,2024,São Paulo Grand Prix,5,"[INTERMEDIATE, INTERMEDIATE, WET, INTERMEDIATE...","[27, 1, 3, 1, 37]",Kick Sauber,19,21.662687,85.258706,...,25.769652,188.955224,0.735821,150.787674,2.288196,117.241552,7.945511,223.628698,8.589053,34.694695


In [5]:
stints_2or3 = stints_transformed[stints_transformed['NumberOfStints'].isin([2, 3])]
stints_2or3

,Driver,Year,Circuit,NumberOfStints,CompoundStrategy,StintLengthStrategy,Team,StartingPosition,AirTemp,Humidity,...,TrackTemp,WindDirection,WindSpeed,mean_brake_time,std_brake_time,AvgSpeed,StdSpeed,AvgSpeedDelta,StdSpeedDelta,AvgGearChanges
1,ALB,2019,Abu Dhabi Grand Prix,2,"[MEDIUM, HARD]","[13, 42]",Red Bull Racing,5,26.018045,64.471429,...,29.566165,221.097744,1.264662,20.155587,1.190986,190.146518,4.104201,246.565018,15.577114,48.759158
2,ALB,2019,Australian Grand Prix,2,"[SOFT, MEDIUM]","[14, 43]",Toro Rosso,13,23.477869,70.453279,...,41.313115,155.327869,1.166393,16.107622,2.045714,207.657799,6.325757,225.931096,19.118794,45.691860
3,ALB,2019,Austrian Grand Prix,2,"[MEDIUM, HARD]","[35, 35]",Toro Rosso,18,34.444068,17.965254,...,50.744068,188.855932,1.225424,13.635129,2.089451,219.187816,6.962654,240.657143,15.165511,38.714286
4,ALB,2019,Azerbaijan Grand Prix,2,"[SOFT, MEDIUM]","[12, 38]",Toro Rosso,11,19.763566,51.408527,...,39.654264,167.480620,1.249612,26.073193,2.424288,193.901751,7.891566,269.989031,16.208025,65.037281
5,ALB,2019,Bahrain Grand Prix,3,"[SOFT, SOFT, MEDIUM]","[9, 16, 32]",Toro Rosso,12,26.210853,53.372093,...,28.610853,60.720930,2.710853,21.511148,2.543971,193.331782,9.221221,258.327546,16.347423,55.267361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,ZHO,2024,Italian Grand Prix,2,"[MEDIUM, HARD]","[15, 37]",Kick Sauber,20,33.101504,32.285714,...,49.311278,200.624060,1.518797,14.722250,2.033569,237.439706,8.250280,271.856757,13.451532,38.919820
2348,ZHO,2024,Mexico City Grand Prix,2,"[HARD, MEDIUM]","[43, 27]",Kick Sauber,19,19.888679,51.050314,...,35.602516,291.352201,2.150943,19.938504,2.499049,179.173394,14.764579,262.885874,14.995403,35.329457
2350,ZHO,2024,Singapore Grand Prix,2,"[HARD, MEDIUM]","[34, 27]",Kick Sauber,20,30.786250,74.650000,...,36.445625,192.493750,0.955000,24.642124,1.980963,176.032457,5.536413,233.619826,12.504193,50.650871
2351,ZHO,2024,Spanish Grand Prix,3,"[SOFT, MEDIUM, HARD]","[9, 32, 24]",Kick Sauber,15,24.132468,63.441558,...,41.096104,207.370130,2.123377,17.048671,2.256758,202.696363,7.376482,225.560185,18.007868,36.703704


In [6]:
corrections = {
    '70th Anniversary Grand Prix': 'Silverstone Circuit',
    'British Grand Prix': 'Silverstone Circuit',
    'São Paulo Grand Prix': 'Interlagos Circuit',
    'Brazilian Grand Prix': 'Interlagos Circuit',
    'Abu Dhabi Grand Prix': 'Yas Marina Circuit',
    'Australian Grand Prix': 'Albert Park Circuit',
    'Azerbaijan Grand Prix': 'Baku City Circuit',
    'Bahrain Grand Prix': 'Bahrain International Circuit - 57 laps',
    'Belgian Grand Prix': 'Circuit de Spa-Francorchamps',
    'Canadian Grand Prix': 'Circuit Gilles Villeneuve',
    'Chinese Grand Prix': 'Shanghai International Circuit',
    'Dutch Grand Prix': 'Circuit Zandvoort',
    'Eifel Grand Prix': 'Nürburgring GP-Strecke Circuit',
    'French Grand Prix': 'Circuit Paul Ricard',
    'Emilia Romagna Grand Prix': 'Autodromo Enzo e Dino Ferrari',
    'German Grand Prix': 'Hockenheimring Circuit',
    'Hungarian Grand Prix': 'Hungaroring Circuit',
    'Italian Grand Prix': 'Autodromo Nazionale Monza',
    'Japanese Grand Prix': 'Suzuka Circuit',
    'Las Vegas Grand Prix': 'Las Vegas Strip Circuit',
    'Mexican Grand Prix': 'Autódromo Hermanos Rodríguez',
    'Mexico City Grand Prix': 'Autódromo Hermanos Rodríguez',
    'Miami Grand Prix': 'Miami International Autodrome',
    'Monaco Grand Prix': 'Circuit de Monaco',
    'Portuguese Grand Prix': 'Algarve International Circuit',
    'Qatar Grand Prix': 'Lusail International Circuit',
    'Russian Grand Prix': 'Sochi Autodrom',
    'Sakhir Grand Prix': 'Bahrain International Circuit - 87 laps',
    'Saudi Arabian Grand Prix': 'Jeddah Corniche Circuit',
    'Singapore Grand Prix': 'Marina Bay Street Circuit',
    'Spanish Grand Prix': 'Circuit de Barcelona-Catalunya',
    'Styrian Grand Prix': 'Red Bull Ring',
    'Austrian Grand Prix': 'Red Bull Ring',
    'Turkish Grand Prix': 'Istanbul Park',
    'Tuscan Grand Prix': 'Autodromo del Mugello',
    'United States Grand Prix': 'Circuit of the Americas',
}

# Create a new DataFrame with corrected circuit names
stints_2or3_corrected = stints_2or3.copy()
stints_2or3_corrected['Circuit'] = stints_2or3['Circuit'].map(corrections)
stints_2or3_corrected

,Driver,Year,Circuit,NumberOfStints,CompoundStrategy,StintLengthStrategy,Team,StartingPosition,AirTemp,Humidity,...,TrackTemp,WindDirection,WindSpeed,mean_brake_time,std_brake_time,AvgSpeed,StdSpeed,AvgSpeedDelta,StdSpeedDelta,AvgGearChanges
1,ALB,2019,Yas Marina Circuit,2,"[MEDIUM, HARD]","[13, 42]",Red Bull Racing,5,26.018045,64.471429,...,29.566165,221.097744,1.264662,20.155587,1.190986,190.146518,4.104201,246.565018,15.577114,48.759158
2,ALB,2019,Albert Park Circuit,2,"[SOFT, MEDIUM]","[14, 43]",Toro Rosso,13,23.477869,70.453279,...,41.313115,155.327869,1.166393,16.107622,2.045714,207.657799,6.325757,225.931096,19.118794,45.691860
3,ALB,2019,Red Bull Ring,2,"[MEDIUM, HARD]","[35, 35]",Toro Rosso,18,34.444068,17.965254,...,50.744068,188.855932,1.225424,13.635129,2.089451,219.187816,6.962654,240.657143,15.165511,38.714286
4,ALB,2019,Baku City Circuit,2,"[SOFT, MEDIUM]","[12, 38]",Toro Rosso,11,19.763566,51.408527,...,39.654264,167.480620,1.249612,26.073193,2.424288,193.901751,7.891566,269.989031,16.208025,65.037281
5,ALB,2019,Bahrain International Circuit - 57 laps,3,"[SOFT, SOFT, MEDIUM]","[9, 16, 32]",Toro Rosso,12,26.210853,53.372093,...,28.610853,60.720930,2.710853,21.511148,2.543971,193.331782,9.221221,258.327546,16.347423,55.267361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,ZHO,2024,Autodromo Nazionale Monza,2,"[MEDIUM, HARD]","[15, 37]",Kick Sauber,20,33.101504,32.285714,...,49.311278,200.624060,1.518797,14.722250,2.033569,237.439706,8.250280,271.856757,13.451532,38.919820
2348,ZHO,2024,Autódromo Hermanos Rodríguez,2,"[HARD, MEDIUM]","[43, 27]",Kick Sauber,19,19.888679,51.050314,...,35.602516,291.352201,2.150943,19.938504,2.499049,179.173394,14.764579,262.885874,14.995403,35.329457
2350,ZHO,2024,Marina Bay Street Circuit,2,"[HARD, MEDIUM]","[34, 27]",Kick Sauber,20,30.786250,74.650000,...,36.445625,192.493750,0.955000,24.642124,1.980963,176.032457,5.536413,233.619826,12.504193,50.650871
2351,ZHO,2024,Circuit de Barcelona-Catalunya,3,"[SOFT, MEDIUM, HARD]","[9, 32, 24]",Kick Sauber,15,24.132468,63.441558,...,41.096104,207.370130,2.123377,17.048671,2.256758,202.696363,7.376482,225.560185,18.007868,36.703704


In [12]:
laps_per_circuit = {
    'Silverstone Circuit': 52,
    'Interlagos Circuit': 71,
    'Yas Marina Circuit': 58, #55,55,58,58,58,58
    'Albert Park Circuit': 58,
    'Baku City Circuit': 51,
    'Bahrain International Circuit - 57 laps': 57,
    'Circuit de Spa-Francorchamps': 44,
    'Circuit Gilles Villeneuve': 70,
    'Shanghai International Circuit': 56,
    'Circuit Zandvoort': 72,
    'Nürburgring GP-Strecke Circuit': 60,
    'Circuit Paul Ricard': 53,
    'Autodromo Enzo e Dino Ferrari': 63,
    'Hockenheimring Circuit': 67,
    'Hungaroring Circuit': 70,
    'Autodromo Nazionale Monza': 53,
    'Suzuka Circuit': 53,
    'Las Vegas Strip Circuit': 50,
    'Autódromo Hermanos Rodríguez': 71,
    'Miami International Autodrome': 57,
    'Circuit de Monaco': 78,
    'Algarve International Circuit': 66,
    'Lusail International Circuit': 57,
    'Sochi Autodrom': 53,
    'Bahrain International Circuit - 87 laps': 87,
    'Jeddah Corniche Circuit': 50,
    'Marina Bay Street Circuit': 61,
    'Circuit de Barcelona-Catalunya': 66,
    'Red Bull Ring': 71,
    'Istanbul Park': 58,
    'Autodromo del Mugello': 59,
    'Circuit of the Americas': 56,
}

# Add 'nLaps' column to the DataFrame based on standardized circuit names
stints_circuit_laps = stints_2or3_corrected.copy()
stints_circuit_laps.insert(3, 'nLaps', stints_circuit_laps['Circuit'].map(laps_per_circuit))
stints_circuit_laps

,Driver,Year,Circuit,nLaps,NumberOfStints,CompoundStrategy,StintLengthStrategy,Team,StartingPosition,AirTemp,...,TrackTemp,WindDirection,WindSpeed,mean_brake_time,std_brake_time,AvgSpeed,StdSpeed,AvgSpeedDelta,StdSpeedDelta,AvgGearChanges
1,ALB,2019,Yas Marina Circuit,58,2,"[MEDIUM, HARD]","[13, 42]",Red Bull Racing,5,26.018045,...,29.566165,221.097744,1.264662,20.155587,1.190986,190.146518,4.104201,246.565018,15.577114,48.759158
2,ALB,2019,Albert Park Circuit,58,2,"[SOFT, MEDIUM]","[14, 43]",Toro Rosso,13,23.477869,...,41.313115,155.327869,1.166393,16.107622,2.045714,207.657799,6.325757,225.931096,19.118794,45.691860
3,ALB,2019,Red Bull Ring,71,2,"[MEDIUM, HARD]","[35, 35]",Toro Rosso,18,34.444068,...,50.744068,188.855932,1.225424,13.635129,2.089451,219.187816,6.962654,240.657143,15.165511,38.714286
4,ALB,2019,Baku City Circuit,51,2,"[SOFT, MEDIUM]","[12, 38]",Toro Rosso,11,19.763566,...,39.654264,167.480620,1.249612,26.073193,2.424288,193.901751,7.891566,269.989031,16.208025,65.037281
5,ALB,2019,Bahrain International Circuit - 57 laps,57,3,"[SOFT, SOFT, MEDIUM]","[9, 16, 32]",Toro Rosso,12,26.210853,...,28.610853,60.720930,2.710853,21.511148,2.543971,193.331782,9.221221,258.327546,16.347423,55.267361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,ZHO,2024,Autodromo Nazionale Monza,53,2,"[MEDIUM, HARD]","[15, 37]",Kick Sauber,20,33.101504,...,49.311278,200.624060,1.518797,14.722250,2.033569,237.439706,8.250280,271.856757,13.451532,38.919820
2348,ZHO,2024,Autódromo Hermanos Rodríguez,71,2,"[HARD, MEDIUM]","[43, 27]",Kick Sauber,19,19.888679,...,35.602516,291.352201,2.150943,19.938504,2.499049,179.173394,14.764579,262.885874,14.995403,35.329457
2350,ZHO,2024,Marina Bay Street Circuit,61,2,"[HARD, MEDIUM]","[34, 27]",Kick Sauber,20,30.786250,...,36.445625,192.493750,0.955000,24.642124,1.980963,176.032457,5.536413,233.619826,12.504193,50.650871
2351,ZHO,2024,Circuit de Barcelona-Catalunya,66,3,"[SOFT, MEDIUM, HARD]","[9, 32, 24]",Kick Sauber,15,24.132468,...,41.096104,207.370130,2.123377,17.048671,2.256758,202.696363,7.376482,225.560185,18.007868,36.703704


In [13]:
# Bahrain International Circuit - 57 laps and Bahrain International Circuit - 87 laps become "Bahrain International Circuit"
stints_bahrain_standardized = stints_circuit_laps.copy()
stints_bahrain_standardized['Circuit'] = stints_circuit_laps['Circuit'].replace({
    'Bahrain International Circuit - 57 laps': 'Bahrain International Circuit',
    'Bahrain International Circuit - 87 laps': 'Bahrain International Circuit'
})
stints_bahrain_standardized

,Driver,Year,Circuit,nLaps,NumberOfStints,CompoundStrategy,StintLengthStrategy,Team,StartingPosition,AirTemp,...,TrackTemp,WindDirection,WindSpeed,mean_brake_time,std_brake_time,AvgSpeed,StdSpeed,AvgSpeedDelta,StdSpeedDelta,AvgGearChanges
1,ALB,2019,Yas Marina Circuit,58,2,"[MEDIUM, HARD]","[13, 42]",Red Bull Racing,5,26.018045,...,29.566165,221.097744,1.264662,20.155587,1.190986,190.146518,4.104201,246.565018,15.577114,48.759158
2,ALB,2019,Albert Park Circuit,58,2,"[SOFT, MEDIUM]","[14, 43]",Toro Rosso,13,23.477869,...,41.313115,155.327869,1.166393,16.107622,2.045714,207.657799,6.325757,225.931096,19.118794,45.691860
3,ALB,2019,Red Bull Ring,71,2,"[MEDIUM, HARD]","[35, 35]",Toro Rosso,18,34.444068,...,50.744068,188.855932,1.225424,13.635129,2.089451,219.187816,6.962654,240.657143,15.165511,38.714286
4,ALB,2019,Baku City Circuit,51,2,"[SOFT, MEDIUM]","[12, 38]",Toro Rosso,11,19.763566,...,39.654264,167.480620,1.249612,26.073193,2.424288,193.901751,7.891566,269.989031,16.208025,65.037281
5,ALB,2019,Bahrain International Circuit,57,3,"[SOFT, SOFT, MEDIUM]","[9, 16, 32]",Toro Rosso,12,26.210853,...,28.610853,60.720930,2.710853,21.511148,2.543971,193.331782,9.221221,258.327546,16.347423,55.267361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,ZHO,2024,Autodromo Nazionale Monza,53,2,"[MEDIUM, HARD]","[15, 37]",Kick Sauber,20,33.101504,...,49.311278,200.624060,1.518797,14.722250,2.033569,237.439706,8.250280,271.856757,13.451532,38.919820
2348,ZHO,2024,Autódromo Hermanos Rodríguez,71,2,"[HARD, MEDIUM]","[43, 27]",Kick Sauber,19,19.888679,...,35.602516,291.352201,2.150943,19.938504,2.499049,179.173394,14.764579,262.885874,14.995403,35.329457
2350,ZHO,2024,Marina Bay Street Circuit,61,2,"[HARD, MEDIUM]","[34, 27]",Kick Sauber,20,30.786250,...,36.445625,192.493750,0.955000,24.642124,1.980963,176.032457,5.536413,233.619826,12.504193,50.650871
2351,ZHO,2024,Circuit de Barcelona-Catalunya,66,3,"[SOFT, MEDIUM, HARD]","[9, 32, 24]",Kick Sauber,15,24.132468,...,41.096104,207.370130,2.123377,17.048671,2.256758,202.696363,7.376482,225.560185,18.007868,36.703704


In [15]:
# remove rows with CompoundStrategy containing 'Intermediate' or 'Wet'
stints_final = stints_bahrain_standardized[
    ~stints_bahrain_standardized['CompoundStrategy'].apply(
        lambda compounds: any(compound in ['INTERMEDIATE', 'WET'] for compound in compounds)
    )
]
stints_final

,Driver,Year,Circuit,nLaps,NumberOfStints,CompoundStrategy,StintLengthStrategy,Team,StartingPosition,AirTemp,...,TrackTemp,WindDirection,WindSpeed,mean_brake_time,std_brake_time,AvgSpeed,StdSpeed,AvgSpeedDelta,StdSpeedDelta,AvgGearChanges
1,ALB,2019,Yas Marina Circuit,58,2,"[MEDIUM, HARD]","[13, 42]",Red Bull Racing,5,26.018045,...,29.566165,221.097744,1.264662,20.155587,1.190986,190.146518,4.104201,246.565018,15.577114,48.759158
2,ALB,2019,Albert Park Circuit,58,2,"[SOFT, MEDIUM]","[14, 43]",Toro Rosso,13,23.477869,...,41.313115,155.327869,1.166393,16.107622,2.045714,207.657799,6.325757,225.931096,19.118794,45.691860
3,ALB,2019,Red Bull Ring,71,2,"[MEDIUM, HARD]","[35, 35]",Toro Rosso,18,34.444068,...,50.744068,188.855932,1.225424,13.635129,2.089451,219.187816,6.962654,240.657143,15.165511,38.714286
4,ALB,2019,Baku City Circuit,51,2,"[SOFT, MEDIUM]","[12, 38]",Toro Rosso,11,19.763566,...,39.654264,167.480620,1.249612,26.073193,2.424288,193.901751,7.891566,269.989031,16.208025,65.037281
5,ALB,2019,Bahrain International Circuit,57,3,"[SOFT, SOFT, MEDIUM]","[9, 16, 32]",Toro Rosso,12,26.210853,...,28.610853,60.720930,2.710853,21.511148,2.543971,193.331782,9.221221,258.327546,16.347423,55.267361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,ZHO,2024,Autodromo Nazionale Monza,53,2,"[MEDIUM, HARD]","[15, 37]",Kick Sauber,20,33.101504,...,49.311278,200.624060,1.518797,14.722250,2.033569,237.439706,8.250280,271.856757,13.451532,38.919820
2348,ZHO,2024,Autódromo Hermanos Rodríguez,71,2,"[HARD, MEDIUM]","[43, 27]",Kick Sauber,19,19.888679,...,35.602516,291.352201,2.150943,19.938504,2.499049,179.173394,14.764579,262.885874,14.995403,35.329457
2350,ZHO,2024,Marina Bay Street Circuit,61,2,"[HARD, MEDIUM]","[34, 27]",Kick Sauber,20,30.786250,...,36.445625,192.493750,0.955000,24.642124,1.980963,176.032457,5.536413,233.619826,12.504193,50.650871
2351,ZHO,2024,Circuit de Barcelona-Catalunya,66,3,"[SOFT, MEDIUM, HARD]","[9, 32, 24]",Kick Sauber,15,24.132468,...,41.096104,207.370130,2.123377,17.048671,2.256758,202.696363,7.376482,225.560185,18.007868,36.703704


In [16]:
stints_final.to_csv("..\\data\\stints_2019-2024.csv", index=False)

In [18]:
import fastf1
session = fastf1.get_session(2022, 'Japanese Grand Prix', 'R')
session.load()

core           INFO 	Loading data for Japanese Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '31', '44', '5', '14', '63', '6', '4', '3', '18', '22', '20', '77', '24', '47', '10', '55', '23']


In [55]:
session.laps.pick_laps([28,29]).pick_drivers(['VER']).get_car_data()['Throttle'].unique() #.pick_laps([30,31]).pick_drivers(['BOT']).get_car_data()


array([100.,   0.,  42.,  45.,  47.,  50.,  59.,  63.,  78.,  86.,   2.,
         9.,  41.,  51.,  10.,  44.,  49.,  55.,  61.,  83.,  90.,  19.,
        29.,  46.,  56.,  67.,  85.,  23.,  39.,  73.,  80.,  91.,  96.,
        25.,  22.,  27.,  76.,  89.,  15.,  36.,  71.,  92.,   5.,  32.,
        34.,  53.,  64.,  70.,  82.,  84.,  98.,  16.,  21.,  30.,  62.,
        87.,   1.,   3.,  40.,  43.,  58.,  60.,  26.,  97.,  17.,  37.,
        18.,  35.,  54.,  72.,  75.,  12.,  14.,  48.,  88.,   8.,  24.,
        38.,  79.,  93.,  52.,  28.,  57.,  20.,  31.,   4.,   6.,   7.,
        11.,  13.])

In [ ]:
# telemetry_laps[telemetry_laps['Throttle'] == 104]